In [8]:
# Pfad zu Markdown File festlegen

#extract = "origin"
extract = "geschichtsbuch"

markdown_path_origin = "docs/extracted_content_dan_brown.md"
markdown_path_path_geschichtsbuch = "docs/extracted_content_geschichtsbuch.md" 

json_chunk_origin = "docs/chunks_dan_brown.json"
json_chunk_geschichtsbuch = "docs/chunks_geschichtsbuch.json"

if extract == "geschichtsbuch": 
    markdown_path = markdown_path_path_geschichtsbuch
    json_chunk_path = json_chunk_geschichtsbuch
else:
    markdown_path = markdown_path_origin
    json_chunk_path = json_chunk_origin

print(f"markdown_path is: {markdown_path}")
print(f"json_chunk_path is: {json_chunk_path}")


markdown_path is: docs/extracted_content_geschichtsbuch.md
json_chunk_path is: docs/chunks_geschichtsbuch.json


## Parameter für Chunking

In [9]:
# Chunking-Parameter
CHUNK_SIZE = 1500
OVERLAP = 200

## Markdown-Datei einlesen

In [10]:
# Markdown-Datei einlesen
with open(markdown_path, "r", encoding="utf-8") as f:
    markdown_text = f.read()

print(f"Datei geladen: {markdown_path}")
print(f"Gesamtlänge: {len(markdown_text)} Zeichen")
print(f"Gesamtlänge: {len(markdown_text.split())} Wörter")

Datei geladen: docs/extracted_content_geschichtsbuch.md
Gesamtlänge: 479526 Zeichen
Gesamtlänge: 74594 Wörter


## Character Splitting implementieren

In [11]:
import re

def extract_page_info(text, chunk_start, chunk_end):
    """
    Extrahiert Seiteninformationen für einen gegebenen Text-Bereich.
    Sucht nach '## Seite X' Markern im Text.
    """
    # Finde alle Seiten-Marker im gesamten Text bis zum Chunk-Ende
    page_pattern = r'## Seite (\d+)'
    pages_found = []
    
    # Finde alle Seiten-Marker mit ihren Positionen
    for match in re.finditer(page_pattern, text[:chunk_end]):
        page_num = int(match.group(1))
        page_pos = match.start()
        pages_found.append((page_num, page_pos))
    
    if not pages_found:
        return None
    
    # Finde die Seite(n), auf der/denen sich der Chunk befindet
    chunk_pages = set()
    
    # Seite am Anfang des Chunks
    for page_num, page_pos in reversed(pages_found):
        if page_pos <= chunk_start:
            chunk_pages.add(page_num)
            break
    
    # Alle Seiten innerhalb des Chunks
    for page_num, page_pos in pages_found:
        if chunk_start <= page_pos < chunk_end:
            chunk_pages.add(page_num)
    
    return sorted(list(chunk_pages)) if chunk_pages else None

def character_split(text, chunk_size, overlap):
    chunks = []
    start = 0
    text_length = len(text)
    
    while start < text_length:
        
        end = start + chunk_size # Ende des Chunks
        
        chunk = text[start:end]
        page_info = extract_page_info(text, start, end)
        
        chunks.append({
            'text': chunk,
            'start': start,
            'end': end,
            'pages': page_info
        })
        
        start = end - overlap # Nächste Startposition: Ende minus Overlap
        
        # Verhindere Endlosschleife bei sehr kleinen Texten
        if start + chunk_size >= text_length and start < text_length:
            # Letzter Chunk
            chunk = text[start:]
            page_info = extract_page_info(text, start, text_length)
            chunks.append({
                'text': chunk,
                'start': start,
                'end': text_length,
                'pages': page_info
            })
            break
    
    return chunks

# Text in Chunks aufteilen
chunks = character_split(markdown_text, CHUNK_SIZE, OVERLAP)

print(f"\n{'='*60}")
print(f"Text wurde in {len(chunks)} Chunks aufgeteilt")
print(f"{'='*60}")
print(f"\nErste 3 Chunks (Vorschau):")
print(f"-"*60)

for i, chunk_data in enumerate(chunks[:3]):
    chunk_text = chunk_data['text']
    pages = chunk_data['pages']
    print(f"\n--- Chunk {i+1} (Länge: {len(chunk_text)} Zeichen, Seite(n): {pages}) ---")
    print(chunk_text[:200] + "..." if len(chunk_text) > 200 else chunk_text)


Text wurde in 369 Chunks aufgeteilt

Erste 3 Chunks (Vorschau):
------------------------------------------------------------

--- Chunk 1 (Länge: 1500 Zeichen, Seite(n): [12]) ---


## Seite 12

Vorwort
Vorliegendes Buch baut auf die 'Schweizer Geschichte " von Dr. Ludwig Suter auf, die sich bei der obligatorischen Einführung in den Sekundär-, Bezirks -, Fortbildungs-, Realschu...

--- Chunk 2 (Länge: 1500 Zeichen, Seite(n): [12, 13]) ---
 Sekundarschüler vorher meist keinen Unterricht in der allgemeinen Geschichte genossen haben und darum einer kurzen Einführung in die Vorgeschichte unseres Landes bedürfen , wodurch sie das Werden und...

--- Chunk 3 (Länge: 1500 Zeichen, Seite(n): [13]) ---
hen Darstellungen gewähren auch einen Rückblick auf die religiösen Verhältnisse nach der Reformation und auf die Zustände der gleichen Periode.
Auch in methodischer Hinsicht habe ich mich bestrebt , d...


## Statistiken der Chunks

In [12]:
# Statistiken über die Chunks
chunk_lengths = [len(chunk['text']) for chunk in chunks]

print(f"Anzahl Chunks: {len(chunks)}")
print(f"Durchschnittliche Chunk-Länge: {sum(chunk_lengths) / len(chunk_lengths):.0f} Zeichen")
print(f"Kürzester Chunk: {min(chunk_lengths)} Zeichen")
print(f"Längster Chunk: {max(chunk_lengths)} Zeichen")

# Zeige Seiteninformationen
pages_with_content = set()
for chunk in chunks:
    if chunk['pages']:
        pages_with_content.update(chunk['pages'])

print(f"\nSeiten mit Inhalt: {sorted(pages_with_content)}")
print(f"Anzahl Seiten: {len(pages_with_content)}")

# Überprüfe mit Tokenizer (falls geladen)
try:
    from transformers import AutoTokenizer
    tokenizer = AutoTokenizer.from_pretrained('intfloat/multilingual-e5-small')
    
    # Teste ersten Chunk mit Prefix
    sample_chunk = f"passage: {chunks[0]['text']}"
    tokens = tokenizer(sample_chunk, return_tensors='pt', truncation=False)
    num_tokens = tokens['input_ids'].shape[1]
    
    print(f"\n{'='*60}")
    print(f"Token-Check für ersten Chunk:")
    print(f"{'='*60}")
    print(f"Zeichen: {len(chunks[0]['text'])}")
    print(f"Tokens: {num_tokens}")
    print(f"Status: {'✅ OK (unter 512)' if num_tokens <= 512 else '❌ ZU LANG'}")
except Exception as e:
    print(f"\nTokenizer nicht verfügbar: {e}")

Anzahl Chunks: 369
Durchschnittliche Chunk-Länge: 1499 Zeichen
Kürzester Chunk: 1126 Zeichen
Längster Chunk: 1500 Zeichen

Seiten mit Inhalt: [12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 

## Chunks speichern

In [13]:
import json
from pathlib import Path

# Dateiname basierend auf dem Quell-Dokument
output_file = json_chunk_path

# Chunks mit Metadaten speichern
chunks_data = {
    "source_file": markdown_path,
    "chunk_size": CHUNK_SIZE,
    "overlap": OVERLAP,
    "total_chunks": len(chunks),
    "chunks": [
        {
            "chunk_id": i,
            "text": chunk['text'],
            "length": len(chunk['text']),
            "pages": chunk['pages'],
            "start_pos": chunk['start'],
            "end_pos": chunk['end']
        }
        for i, chunk in enumerate(chunks)
    ]
}

# Als JSON speichern
with open(output_file, "w", encoding="utf-8") as f:
    json.dump(chunks_data, f, ensure_ascii=False, indent=2)

print(f"✅ {len(chunks)} Chunks gespeichert in: {output_file}")
print(f"Dateigröße: {Path(output_file).stat().st_size / 1024:.1f} KB")
print(f"\nBeispiel Chunk-Metadaten:")
print(json.dumps(chunks_data['chunks'][0], ensure_ascii=False, indent=2)[:300] + "...")

✅ 369 Chunks gespeichert in: docs/chunks_geschichtsbuch.json
Dateigröße: 615.8 KB

Beispiel Chunk-Metadaten:
{
  "chunk_id": 0,
  "text": "\n\n## Seite 12\n\nVorwort\nVorliegendes Buch baut auf die 'Schweizer Geschichte \" von Dr. Ludwig Suter auf, die sich bei der obligatorischen Einführung in den Sekundär-, Bezirks -, Fortbildungs-, Realschulen , Gymnasien und Lehrerseminare vor züglich bewährt hat.\nIm ...


## Chunks wieder einlesen (zum Testen)

In [14]:
# Chunks wieder einlesen
with open(output_file, "r", encoding="utf-8") as f:
    loaded_data = json.load(f)

# Metadaten anzeigen
print("Geladene Daten:")
print(f"  Quelle: {loaded_data['source_file']}")
print(f"  Chunk-Größe: {loaded_data['chunk_size']}")
print(f"  Overlap: {loaded_data['overlap']}")
print(f"  Anzahl Chunks: {loaded_data['total_chunks']}")

# Die eigentlichen Text-Chunks extrahieren
loaded_chunks = [chunk_data["text"] for chunk_data in loaded_data["chunks"]]

print(f"\n✅ {len(loaded_chunks)} Chunks erfolgreich geladen")
print(f"\nBeispiel - Erster Chunk (erste 150 Zeichen):")
print(loaded_chunks[0][:150] + "...")
print(f"\nSeiteninformationen für erste 3 Chunks:")
for i in range(min(3, len(loaded_data['chunks']))):
    chunk_info = loaded_data['chunks'][i]
    print(f"  Chunk {i+1}: Seite(n) {chunk_info['pages']}")

Geladene Daten:
  Quelle: docs/extracted_content_geschichtsbuch.md
  Chunk-Größe: 1500
  Overlap: 200
  Anzahl Chunks: 369

✅ 369 Chunks erfolgreich geladen

Beispiel - Erster Chunk (erste 150 Zeichen):


## Seite 12

Vorwort
Vorliegendes Buch baut auf die 'Schweizer Geschichte " von Dr. Ludwig Suter auf, die sich bei der obligatorischen Einführung in...

Seiteninformationen für erste 3 Chunks:
  Chunk 1: Seite(n) [12]
  Chunk 2: Seite(n) [12, 13]
  Chunk 3: Seite(n) [13]
